# Large Data Cleaning Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

In [95]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Import data the large data set from csv.

In [97]:
flights = pd.read_csv("data/large/flights_7_mil.csv")

In [98]:
flights.columns

Index(['Unnamed: 0', 'fl_date', 'mkt_unique_carrier', 'branded_code_share',
       'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [99]:
#Drop unnecessary columns
flights = flights[[
                    'fl_date', 'mkt_unique_carrier','cancelled','diverted',
                    'crs_elapsed_time','crs_dep_time','crs_arr_time',
                    'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay','late_aircraft_delay',
                    'dep_delay','arr_delay',
                   ]]

In [100]:
flights.isnull().sum()

fl_date                      0
mkt_unique_carrier           0
cancelled                    0
diverted                     0
crs_elapsed_time            19
crs_dep_time                 0
crs_arr_time                 0
carrier_delay          5723546
weather_delay          5723546
nas_delay              5723546
security_delay         5723546
late_aircraft_delay    5723546
dep_delay               113102
arr_delay               133431
dtype: int64

In [101]:
test = flights[(flights['cancelled'] != 1)&(flights['diverted']!=1)]

In [102]:
test = test[(test['crs_elapsed_time'].notnull())]
test.isnull().sum()

fl_date                      0
mkt_unique_carrier           0
cancelled                    0
diverted                     0
crs_elapsed_time             0
crs_dep_time                 0
crs_arr_time                 0
carrier_delay          5591085
weather_delay          5591085
nas_delay              5591085
security_delay         5591085
late_aircraft_delay    5591085
dep_delay                 1818
arr_delay                  970
dtype: int64

In [103]:
def fill_columns(fill,columns,df):
    for col in columns:
        df[col].fillna(fill,inplace=True)

In [104]:
# Fill all null delays columns with 0.
columns = ['late_aircraft_delay','security_delay','nas_delay','weather_delay','carrier_delay','dep_delay','arr_delay']
fill_columns(0,columns,test)

In [105]:
test.isnull().sum()

fl_date                0
mkt_unique_carrier     0
cancelled              0
diverted               0
crs_elapsed_time       0
crs_dep_time           0
crs_arr_time           0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
dep_delay              0
arr_delay              0
dtype: int64

In [106]:
test['fl_date'] = pd.to_datetime(test['fl_date'],format='%Y-%m-%d')
test['year'] = pd.DatetimeIndex(test['fl_date']).year
test['month'] = pd.DatetimeIndex(test['fl_date']).month
test['day'] = pd.DatetimeIndex(test['fl_date']).day
test = test.drop(columns=['fl_date','cancelled','diverted'])

In [107]:
test.head()

,mkt_unique_carrier,crs_elapsed_time,crs_dep_time,crs_arr_time,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,dep_delay,arr_delay,year,month,day
0,AA,104.0,1837,2021,2.0,0.0,0.0,0.0,35.0,50.0,37.0,2019,8,27
1,AA,142.0,709,931,0.0,0.0,46.0,0.0,0.0,-5.0,46.0,2019,8,27
2,AA,152.0,1012,1244,8.0,0.0,1.0,0.0,45.0,53.0,54.0,2019,8,27
3,AA,303.0,1538,2341,0.0,0.0,0.0,0.0,0.0,-1.0,-21.0,2019,8,27
4,AA,90.0,1007,1137,0.0,0.0,0.0,0.0,56.0,69.0,56.0,2019,8,27


In [108]:
test.select_dtypes(include='object').columns

Index(['mkt_unique_carrier'], dtype='object')

In [109]:
# test = pd.get_dummies(test, columns=test.select_dtypes(include='object').columns)
test.head()

,mkt_unique_carrier,crs_elapsed_time,crs_dep_time,crs_arr_time,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,dep_delay,arr_delay,year,month,day
0,AA,104.0,1837,2021,2.0,0.0,0.0,0.0,35.0,50.0,37.0,2019,8,27
1,AA,142.0,709,931,0.0,0.0,46.0,0.0,0.0,-5.0,46.0,2019,8,27
2,AA,152.0,1012,1244,8.0,0.0,1.0,0.0,45.0,53.0,54.0,2019,8,27
3,AA,303.0,1538,2341,0.0,0.0,0.0,0.0,0.0,-1.0,-21.0,2019,8,27
4,AA,90.0,1007,1137,0.0,0.0,0.0,0.0,56.0,69.0,56.0,2019,8,27


In [110]:
# fill Max column with the largest value of delay column name
# only flights with delays > 0

test['Max'] = test[test['arr_delay']>0][['late_aircraft_delay','security_delay',
                         'nas_delay','weather_delay','carrier_delay']].idxmax(axis=1)

# drop the delay columns
test = test.drop(columns = ['late_aircraft_delay','security_delay',
                            'nas_delay','weather_delay','carrier_delay',
                            'mkt_unique_carrier'])
test.head()

,crs_elapsed_time,crs_dep_time,crs_arr_time,dep_delay,arr_delay,year,month,day,Max
0,104.0,1837,2021,50.0,37.0,2019,8,27,late_aircraft_delay
1,142.0,709,931,-5.0,46.0,2019,8,27,nas_delay
2,152.0,1012,1244,53.0,54.0,2019,8,27,late_aircraft_delay
3,303.0,1538,2341,-1.0,-21.0,2019,8,27,NaN
4,90.0,1007,1137,69.0,56.0,2019,8,27,late_aircraft_delay


In [111]:
test.fillna('early',inplace=True)

In [112]:
sample = test.sample(100000,weights=test.groupby('Max')['Max'].transform('count'), random_state=123)

In [113]:
sample.groupby('Max')['Max'].count()

Max
carrier_delay            508
early                  87932
late_aircraft_delay    10918
nas_delay                630
weather_delay             12
Name: Max, dtype: int64

In [114]:
test.groupby('Max')['Max'].count()

Max
carrier_delay           334773
early                  4512541
late_aircraft_delay    1577042
nas_delay               400003
security_delay            2227
weather_delay            40952
Name: Max, dtype: int64

In [115]:
# Get dummies for the delay columns
test = pd.get_dummies(test, prefix='', prefix_sep='')
test.shape

(6867538, 14)

Export the dataframe to csv.

In [116]:
test.to_csv('data/large/flights_7mil_encoded_delays_no_catagorical_sub.csv')